SQL at Scale with Spark SQL
Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries.

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the country_club database.


1. The __`bookings`__ table, 2. The __`facilities`__ table, and 3. The __`members`__ table.
You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable, please download this notebook as an IPython notebook (File -> Export -> IPython Notebook) and upload it to your GitHub.

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: string (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: string (nullable = true)

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


# Question and Answers are given below

Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities where membercost > 0 limit 100; 

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select COUNT(*) from facilities where membercost == 0 limit 100; 

count(1)
4


Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost?¶

Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql

select facid, name, membercost, monthlymaintenance
from facilities
where membercost > 0 and ((membercost/monthlymaintenance) * 100) < 20
limit 10


facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities
where name like '%2';

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100?¶

Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql

select name, monthlymaintenance,
    (case when monthlymaintenance<100 then 'cheap'
    when monthlymaintenance>100 then 'expensive'
    end) as cheap_or_expensive
from facilities


name,monthlymaintenance,cheap_or_expensive
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select firstname, surname as lastname from Members
where memid > 25
order by memid desc;


firstname,lastname
Darren,Smith
Erica,Crumpet
John,Hunt
Hyacinth,Tupperware
Millicent,Purview
Henry,Worthington-Smyth
David,Farrell
Henrietta,Rumney
Douglas,Jones


Q7: How can you produce a list of all members who have used a tennis court?

Include in your output the name of the court, and the name of the member formatted as a single column.

Ensure no duplicate data

Also order by the member name.

In [0]:
%sql
select f.name, m.firstname, m.surname
from Bookings as b
inner join facilities as f
on b.facid = f.facid
inner join Members as m
on m.memid = b.memid
where b.facid == 0 or b.facid == 1
group by f.name, m.firstname,m.surname
order by firstname, surname;

name,firstname,surname
Tennis Court 2,Anne,Baker
Tennis Court 1,Anne,Baker
Tennis Court 1,Burton,Tracy
Tennis Court 2,Burton,Tracy
Tennis Court 1,Charles,Owen
Tennis Court 2,Charles,Owen
Tennis Court 2,Darren,Smith
Tennis Court 2,David,Farrell
Tennis Court 1,David,Farrell
Tennis Court 1,David,Jones


Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30?

  Remember that guests have different costs to members (the listed costs are per half-hour 'slot')

  The guest user's ID is always 0.

Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

  Order by descending cost, and do not use any subqueries.

In [0]:
%sql

select cast(b.starttime as TIMESTAMP), CONCAT(m.firstname, " ", m.surname) as subscriber_name, f.name,
 (case when b.memid=0 and ((b.slots * f.guestcost) + f.monthlymaintenance) > 30 then ((b.slots * f.guestcost) + f.monthlymaintenance) 
 when b.memid>0 and ((b.slots * f.membercost)+ f.monthlymaintenance) > 30 then ((b.slots * f.membercost)+ f.monthlymaintenance) end) as total_cost
from Bookings as b
inner join facilities as f
on b.facid = f.facid
inner join Members as m
on m.memid = b.memid
where (starttime Like '2012-09-14%') 
group by b.starttime, subscriber_name, f.name, total_cost
having total_cost is not null
order by total_cost Desc;

starttime,subscriber_name,name,total_cost
2012-09-14T11:00:00.000+0000,GUEST GUEST,Massage Room 2,3320.0
2012-09-14T13:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T16:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T09:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T14:00:00.000+0000,Jemima Farrell,Massage Room 1,3039.6
2012-09-14T09:30:00.000+0000,Florence Bader,Massage Room 2,3019.8
2012-09-14T08:00:00.000+0000,Jack Smith,Massage Room 1,3019.8
2012-09-14T18:00:00.000+0000,Burton Tracy,Massage Room 1,3019.8
2012-09-14T11:00:00.000+0000,Jemima Farrell,Massage Room 1,3019.8
2012-09-14T12:00:00.000+0000,Ponder Stibbons,Massage Room 1,3019.8


Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql

select sub.starttime, sub.subscriber_name, sub.customer, sub.total_cost
      
from 
    
    (select cast(b.starttime as TIMESTAMP), CONCAT(m.firstname, " ", m.surname) as subscriber_name, f.name as customer,
            (case when b.memid=0 and ((b.slots * f.guestcost) + f.monthlymaintenance) > 30 then ((b.slots * f.guestcost) + f.monthlymaintenance) 
                  when b.memid>0 and ((b.slots * f.membercost)+ f.monthlymaintenance) > 30 then ((b.slots * f.membercost)+ f.monthlymaintenance) end) as total_cost
     from Bookings as b
     inner join facilities as f
     on b.facid = f.facid
     inner join Members as m
     on m.memid = b.memid
     where (b.starttime Like '2012-09-14%') 
    ) as sub
where (sub.starttime Like '2012-09-14%')
group by sub.starttime, sub.subscriber_name, sub.customer, sub.total_cost
having sub.total_cost is not null
order by sub.total_cost Desc;
    

starttime,subscriber_name,customer,total_cost
2012-09-14T11:00:00.000+0000,GUEST GUEST,Massage Room 2,3320.0
2012-09-14T13:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T16:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T09:00:00.000+0000,GUEST GUEST,Massage Room 1,3160.0
2012-09-14T14:00:00.000+0000,Jemima Farrell,Massage Room 1,3039.6
2012-09-14T11:00:00.000+0000,Jemima Farrell,Massage Room 1,3019.8
2012-09-14T18:00:00.000+0000,Burton Tracy,Massage Room 1,3019.8
2012-09-14T08:00:00.000+0000,Jack Smith,Massage Room 1,3019.8
2012-09-14T12:00:00.000+0000,Ponder Stibbons,Massage Room 1,3019.8
2012-09-14T09:30:00.000+0000,Florence Bader,Massage Room 2,3019.8


Q10: Produce a list of facilities with a total revenue less than 1000.

The output should have facility name and total revenue, sorted by revenue.

Remember that there's a different cost for guests and members!

In [0]:
%sql

select subq.facility_name, ROUND(SUM(subq.total_cost),2) as total_revenue
from 
     (select CONCAT(m.firstname, " ", m.surname) as subscriber_name, f.name as facility_name, 
            (case when b.memid=0 then ((b.slots * f.guestcost) + f.monthlymaintenance) 
                  when b.memid>0 then ((b.slots * f.membercost)+ f.monthlymaintenance) end) as total_cost
      from Bookings as b
      inner join facilities as f
      on b.facid = f.facid
      inner join Members as m
      on m.memid = b.memid
      ) as subq
group by subq.facility_name
order by total_revenue;

facility_name,total_revenue
Table Tennis,4210.0
Snooker Table,6900.0
Pool Table,12810.0
Badminton Court,21056.5
Squash Court,48668.0
Tennis Court 2,92110.0
Tennis Court 1,95460.0
Massage Room 2,347454.6
Massage Room 1,1937351.6
